In [2]:
import numpy as np
import pandas as pd
import ast

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


<h1 align='center'>Data Preporccessing<h1>

### Step 1: import Datasets

In [9]:
df1 = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
df2 = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
print(df1.shape)
print(df2.shape)

(4803, 20)
(4803, 4)


### Step 2: Combining Datasets

In [4]:
df1.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
df2.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


we have to convert `movie_id` to `id` before merging these 2 dataset

In [10]:
df2['id'] = df2['movie_id']

In [8]:
df2.head(1)

,movie_id,title,cast,crew,id,movie_title
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",19995,Avatar


now combine these two dataset with respect to `id`

In [ ]:
df = df1.merge(df2, on='id')
print(df.shape)
print(df.info())

In [ ]:
# run these one by one to see the data 
df.nunique()
df['status'].value_counts()

In [57]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### Step 3: Taking only relevelnt column

In [12]:
movies = df[['id','title_x','overview','genres',
             'keywords','original_language','production_companies',
             'tagline','cast','crew']]


In [117]:
movies.head(1)

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## Handling missing values

`movies.isnull().sum()` will show the total number of null values in each column

In [13]:
movies.isnull().sum()

id                        0
title_x                   0
overview                  3
genres                    0
keywords                  0
original_language         0
production_companies      0
tagline                 844
cast                      0
crew                      0
dtype: int64

#### `overview` column

In [119]:
movies['overview'].isnull().sum()

3

In [14]:
movies['overview'] = movies['overview'].fillna(" ")

/tmp/ipykernel_37/1190669893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['overview'] = movies['overview'].fillna(" ")


In [15]:
movies['overview'].isnull().sum()

0

#### `tagline` column

In [122]:
movies['tagline'].isnull().sum()

844

In [16]:
movies['tagline'] = movies['tagline'].fillna(" ")

/tmp/ipykernel_37/563664844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tagline'] = movies['tagline'].fillna(" ")


In [124]:
movies['tagline'].isnull().sum()

0

In [17]:
movies.isnull().sum()

id                      0
title_x                 0
overview                0
genres                  0
keywords                0
original_language       0
production_companies    0
tagline                 0
cast                    0
crew                    0
dtype: int64

# Feature engeneering

In [165]:
movies.head(1)

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


#### Making a funtion to convert `genres`, `keywords`

In [18]:
def convert_genres(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(f"gen_{i['name']}")
    return ", ".join(List)

def convert_keywords(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    return ", ".join(List)

'''
# to-do
1. convert in lower case
2. remove space
3. replace comma with space
'''


In [19]:
movies['genres'] = movies['genres'].apply(convert_genres)

/tmp/ipykernel_37/2319996759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = movies['genres'].apply(convert_genres)


In [21]:
movies['keywords'] = movies['keywords'].apply(convert_keywords)

/tmp/ipykernel_37/725424263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(convert_keywords)


In [22]:
movies.head(1)

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","gen_Action, gen_Adventure, gen_Fantasy, gen_Sc...","culture clash, future, space war, space colony...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### Worling on `original language`

In [152]:
movies['original_language'] = "ol_" + movies['original_language'].astype(str)

/tmp/ipykernel_37/4160749926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['original_language'] = "ol_" + movies['original_language'].astype(str)


In [171]:
movies.head(1)

,id,title_x,overview,genres,keywords,original_language,production_companies,tagline,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","gen_Action, gen_Adventure, gen_Fantasy, gen_Sc...","gen_culture clash, gen_future, gen_space war, ...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [157]:
movies['genres'] = "gen_" + movies['genres'].astype(str)

/tmp/ipykernel_37/1292290195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = "gen_" + movies['genres'].astype(str)


In [151]:
# checking production companies
print(movies.iloc[3].title_x)
print(movies.iloc[3].production_companies)

# so we will need all production companies name

The Dark Knight Rises
[{"name": "Legendary Pictures", "id": 923}, {"name": "Warner Bros.", "id": 6194}, {"name": "DC Entertainment", "id": 9993}, {"name": "Syncopy", "id": 9996}]


In [75]:
def convert_str(obj):
    List = []
      
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    return ",".join(List)
